In [1]:
%matplotlib notebook

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy import stats
import requests
import gmaps
import os
import json
from opencage.geocoder import OpenCageGeocode

In [3]:
#Load FEMA Data Sets
owner_data = pd.read_csv('Data Sets/Owners Export_cleaned.csv', encoding='cp1252')
renters_data = pd.read_csv('Data Sets/Renters Export_cleaned.csv')

In [4]:
#Visualize Dataframes
owner_data.head(5)

,Disaster,State,County,City,Zip Code,Valid Registrations,Average FEMA Inspected Damage,Total Inspected,Total Damage,No FEMA Inspected Damage,...,"FEMA Inspected Damage > $30,000",Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
0,4447,OH,Auglaize (County),CRIDERSVILLE,45806,1,$-,1,$-,1,...,0,0,$0,$0,$0,$0,0,0,0,0
1,4447,OH,Auglaize (County),NEW KNOXVILLE,45871,1,"$2,376.23",1,"$2,376.23",0,...,0,1,$697.52,$698,$0,$0,1,0,0,0
2,4447,OH,Auglaize (County),SAINT MARYS,45885,3,$-,3,$-,3,...,0,0,$0,$0,$0,$0,0,0,0,0
3,4447,OH,Auglaize (County),WAPAKONETA,45895,21,"$6,735.89",14,"$94,302.44",4,...,1,5,"$56,475.44","$41,315","$2,406","$12,754",3,1,1,0
4,4447,OH,Darke (County),ARCANUM,45304,1,$-,1,$-,1,...,0,0,$0,$0,$0,$0,0,0,0,0


In [5]:
#Visualize Dataframes
renters_data.head(5)

,Disaster,State,County,City,Zip Code,Valid Registrations,Total Inspected,Inspected with No Damage,Total with Moderate Damage,Total with Major Damage,Total with Substantial Damage,Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
0,4454,TX,Cameron (County),BROWNSVILLE,78521,2,0,0,0,0,0,0,$-,$-,$-,$0,0,0,0,0
1,4454,TX,Cameron (County),BROWNSVILLE,78526,1,1,1,0,0,0,0,$-,$-,$-,$0,0,0,0,0
2,4454,TX,Cameron (County),CAMERON,76520,1,1,1,0,0,0,0,$-,$-,$-,$0,0,0,0,0
3,4454,TX,Cameron (County),COMBES,78535,7,7,6,1,0,0,2,"$4,197.14",$-,"$1,436.00","$2,761",2,0,0,0
4,4454,TX,Cameron (County),HARLINGEN,78550,224,179,110,69,0,0,94,"$227,526.33",$-,"$102,614.12","$124,912",94,0,0,0


In [6]:
#Make copy and clean data sets
converted_renters_data = renters_data.copy()
converted_renters_data['Total Approved IHP Amount'] = converted_renters_data['Total Approved IHP Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_renters_data['Rental Amount'] = converted_renters_data['Rental Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_renters_data['Repair/Replace Amount'] = converted_renters_data['Repair/Replace Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_renters_data['Other Needs Amount'] = converted_renters_data['Other Needs Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')




In [7]:
converted_renters_data.head(5)

,Disaster,State,County,City,Zip Code,Valid Registrations,Total Inspected,Inspected with No Damage,Total with Moderate Damage,Total with Major Damage,Total with Substantial Damage,Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
0,4454,TX,Cameron (County),BROWNSVILLE,78521,2,0,0,0,0,0,0,0.00,0.0,0.00,0.0,0,0,0,0
1,4454,TX,Cameron (County),BROWNSVILLE,78526,1,1,1,0,0,0,0,0.00,0.0,0.00,0.0,0,0,0,0
2,4454,TX,Cameron (County),CAMERON,76520,1,1,1,0,0,0,0,0.00,0.0,0.00,0.0,0,0,0,0
3,4454,TX,Cameron (County),COMBES,78535,7,7,6,1,0,0,2,4197.14,0.0,1436.00,2761.0,2,0,0,0
4,4454,TX,Cameron (County),HARLINGEN,78550,224,179,110,69,0,0,94,227526.33,0.0,102614.12,124912.0,94,0,0,0


In [8]:
#Makes copy and clean data set
converted_owner_data = owner_data.copy()
converted_owner_data["Total Approved IHP Amount"] = converted_owner_data["Total Approved IHP Amount"].str.replace(',','').str.replace('$','').astype('float')
converted_owner_data['Average FEMA Inspected Damage'] = converted_owner_data["Average FEMA Inspected Damage"].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Rental Amount'] = converted_owner_data['Rental Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Repair/Replace Amount'] = converted_owner_data['Repair/Replace Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Other Needs Amount'] = converted_owner_data['Other Needs Amount'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Approved between $1 and $10,000'] = converted_owner_data['Approved between $1 and $10,000'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Approved between $10,001 and $25,000'] = converted_owner_data['Approved between $10,001 and $25,000'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Approved between $25,001 and Max'] = converted_owner_data['Approved between $25,001 and Max'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Total Max Grants'] = converted_owner_data['Total Max Grants'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')
converted_owner_data['Total Damage'] = converted_owner_data['Total Damage'].str.replace(',','').str.replace('$','').str.replace('-','0').astype('float')


In [9]:
converted_owner_data.head(5)

,Disaster,State,County,City,Zip Code,Valid Registrations,Average FEMA Inspected Damage,Total Inspected,Total Damage,No FEMA Inspected Damage,...,"FEMA Inspected Damage > $30,000",Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
0,4447,OH,Auglaize (County),CRIDERSVILLE,45806,1,0.00,1,0.00,1,...,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4447,OH,Auglaize (County),NEW KNOXVILLE,45871,1,2376.23,1,2376.23,0,...,0,1,697.52,698.0,0.0,0.0,1.0,0.0,0.0,0.0
2,4447,OH,Auglaize (County),SAINT MARYS,45885,3,0.00,3,0.00,3,...,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4447,OH,Auglaize (County),WAPAKONETA,45895,21,6735.89,14,94302.44,4,...,1,5,56475.44,41315.0,2406.0,12754.0,3.0,1.0,1.0,0.0
4,4447,OH,Darke (County),ARCANUM,45304,1,0.00,1,0.00,1,...,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
#Count the number of inputs by state
state_counts_rent = converted_renters_data["State"].value_counts()
state_counts_rent.head(5)

FL    5774
PR    4638
TX    3287
NC    1385
MP     999
Name: State, dtype: int64

In [11]:
#Count the number of inputs by state
state_counts_owners = converted_owner_data["State"].value_counts()
state_counts_owners.head(5)

PR    7649
FL    4878
TX    3919
NC    1483
LA    1147
Name: State, dtype: int64

In [12]:
# Using GroupBy in order to separate the data into fields according to "state" values
grouped_renters_df = converted_renters_data.groupby(['State'])

# The object returned is a "GroupBy" object and cannot be viewed normally...
print(grouped_renters_df)

# In order to be visualized, a data function must be used...
grouped_renters_df.count().head(5)

,Disaster,County,City,Zip Code,Valid Registrations,Total Inspected,Inspected with No Damage,Total with Moderate Damage,Total with Major Damage,Total with Substantial Damage,Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
State,,,,,,,,,,,,,,,,,,,
AK,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40
AL,162,162,162,162,162,162,162,162,162,162,162,162,162,162,162,162,162,162,162
AR,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259,259
AS,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14
CA,619,619,619,619,619,619,619,619,619,619,619,619,619,619,619,619,619,619,619


In [13]:
# Using GroupBy in order to separate the data into fields according to "state" values
grouped_owners_df = converted_owner_data.groupby(['State'])

# The object returned is a "GroupBy" object and cannot be viewed normally...
print(grouped_owners_df)

# In order to be visualized, a data function must be used...
grouped_owners_df.count().head(5)

,Disaster,County,City,Zip Code,Valid Registrations,Average FEMA Inspected Damage,Total Inspected,Total Damage,No FEMA Inspected Damage,"FEMA Inspected Damage between $1 and $10,000",...,"FEMA Inspected Damage > $30,000",Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
State,,,,,,,,,,,,,,,,,,,,,
AK,69,69,69,69,69,69,69,69,69,69,...,69,69,69,69,69,69,69,69,69,69
AL,230,230,230,230,230,230,230,230,230,230,...,230,230,230,230,230,230,230,230,230,230
AR,435,435,435,435,435,435,435,435,435,435,...,435,435,435,435,435,435,435,435,435,435
AS,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
CA,654,654,654,654,654,654,654,654,654,654,...,654,654,654,654,654,654,654,654,654,654


In [14]:
#Create a dataframe
count = grouped_owners_df['Disaster'].count()
total = grouped_owners_df['Total Approved IHP Amount'].sum()
grouped_owners_df['Total Approved IHP Amount'].sum()
state_summary_table_owners = pd.DataFrame({"Number of Claims": count,
                                    "Total Approved IHP Amount": total})

In [15]:
#Show the dataframe
state_summary_table_owners.head(5)

,Number of Claims,Total Approved IHP Amount
State,,
AK,69,29581739.00
AL,230,17091133.53
AR,435,16469696.80
AS,19,18607987.67
CA,654,81247028.30


In [16]:
#Create new datafram
count1 = grouped_renters_df['Disaster'].count()
total1 = grouped_renters_df['Total Approved IHP Amount'].sum()
grouped_renters_df['Total Approved IHP Amount'].sum()
state_summary_table_renters = pd.DataFrame({"Number of Claims": count1,
                                    "Total Approved IHP Amount": total1})

In [17]:
#Prep for merger
state_summary_table_combined = state_summary_table_renters.copy()

In [18]:
#Combine two dfs renters and owners
state_summary_table_combined ['Number of Claims Owners'] = state_summary_table_owners['Number of Claims']
state_summary_table_combined ['Total Approved IHP Amount Owners'] = state_summary_table_owners['Total Approved IHP Amount']
state_summary_table_combined ['Number of Claims'] = state_summary_table_owners['Number of Claims'] + state_summary_table_combined ['Number of Claims Owners']
state_summary_table_combined ['Total Approved IHP Amount'] = state_summary_table_owners['Total Approved IHP Amount'] + state_summary_table_combined ['Total Approved IHP Amount Owners']
state_summary_table_combined = state_summary_table_combined.drop(['Number of Claims Owners','Total Approved IHP Amount Owners'], axis=1)

In [19]:
#Show merged table
state_summary_table_combined.head()

,Number of Claims,Total Approved IHP Amount
State,,
AK,138,5.916348e+07
AL,460,3.418227e+07
AR,870,3.293939e+07
AS,38,3.721598e+07
CA,1308,1.624941e+08


In [20]:
#Create combined df and groupby state and couhty
converted_owner_data_slimed = converted_owner_data[['State','County','Total Approved IHP Amount']]
converted_renters_data_slimed = converted_owner_data[['State','County','Total Approved IHP Amount']]
converted_combined_data_slimed = pd.merge(converted_owner_data_slimed, converted_renters_data_slimed, how='outer' )
converted_combined_data_slimed = converted_combined_data_slimed.drop_duplicates(subset=None, keep='first', inplace=False)
county_summary_table = converted_combined_data_slimed.groupby(['State','County']).sum()

In [21]:
#Visualize df
county_summary_table
couny_summary_table['']

Total Approved IHP Amount
State County                                                                       
AK    Alaska Gateway Regional Educational Attendance ...                   68623.95
      Anchorage (Borough)                                               19801204.61
      Kenai Peninsula (Borough)                                           587825.63
      Lower Yukon Regional Educational Attendance Area                    445503.04
      Matanuska-Susitna (Borough)                                        6217654.42
...                                                                             ...
WV    Summers (County)                                                    376390.06
      Webster (County)                                                   2405660.11
      Wetzel (County)                                                     887050.49
WY    Johnson (County)                                                    101064.34
      Niobrara (County)                                                  1072226.06

[870 rows x 1 columns]

In [22]:
#Reset index so we can use state in loop
county_summary_table_a = county_summary_table.reset_index()
a = county_summary_table_a
a

,State,County,Total Approved IHP Amount
0,AK,Alaska Gateway Regional Educational Attendance...,68623.95
1,AK,Anchorage (Borough),19801204.61
2,AK,Kenai Peninsula (Borough),587825.63
3,AK,Lower Yukon Regional Educational Attendance Area,445503.04
4,AK,Matanuska-Susitna (Borough),6217654.42
...,...,...,...
865,WV,Summers (County),376390.06
866,WV,Webster (County),2405660.11
867,WV,Wetzel (County),887050.49
868,WY,Johnson (County),101064.34


In [23]:
#Enter Open Cage key and import it into function
key = ""
geocoder = OpenCageGeocode(key)

In [39]:
#Build loop to request lat and long from Geocoder

#List to append info into
lat = []
lng = []
query = []

#loops through data frame takes county and state and request info
for index, row in a.iterrows():
    if row['State'] == 'PR':
        query = "u'" + row['County'] + ", " + row['State'] + " ,PRI'"
        results = geocoder.geocode(query)
        #Saves lat and long into list
        lat.append(results[0]['geometry']['lat']) 
        lng.append(results[0]['geometry']['lng'])
    else:
        query = "u'" + row['County'] + ", " + row['State'] + " ,US'"
        results = geocoder.geocode(query)
        #Saves lat and long into list
        lat.append(results[0]['geometry']['lat']) 
        lng.append(results[0]['geometry']['lng'])
    


RateLimitExceededError: Your rate limit has expired. It will reset to 2500 on 2020-01-14T00:00:00

In [25]:
#Enter latitude into df
county_summary_table_a['lat'] = lat

In [26]:
#Enter longitude into df
county_summary_table_a['lng'] = lng

In [40]:
county_summary_table_a.head(10)

,State,County,Total Approved IHP Amount,lat,lng
0,AK,Alaska Gateway Regional Educational Attendance...,68623.95,64.000280,-150.000280
1,AK,Anchorage (Borough),19801204.61,64.000280,-150.000280
2,AK,Kenai Peninsula (Borough),587825.63,64.000280,-150.000280
3,AK,Lower Yukon Regional Educational Attendance Area,445503.04,64.000280,-150.000280
4,AK,Matanuska-Susitna (Borough),6217654.42,61.543610,-150.512500
5,AK,Yukon Flats Regional Educational Attendance Area,362321.05,61.124646,-146.265185
6,AK,Yukon Koyukuk Regional Educational Attendance ...,2098606.30,64.839012,-147.784723
7,AL,Baldwin (County),7719236.56,31.294218,-86.381475
8,AL,Blount (County),155485.76,31.294218,-86.381475
9,AL,Calhoun (County),181921.28,31.294218,-86.381475


In [51]:
county_summary_table_aid = county_summary_table_a.sort_values(['Total Approved IHP Amount'], ascending=False)
county_summary_table_aid['Total Approved IHP Amount'] = county_summary_table_aid['Total Approved IHP Amount'].map('${:,.2f}'.format)
county_summary_table_aid.head(10)

,State,County,Total Approved IHP Amount,lat,lng
786,TX,Harris (County),"$635,035,082.75",31.087212,-104.271552
309,LA,East Baton Rouge (Parish),"$308,355,417.88",30.481755,-91.165701
322,LA,Livingston (Parish),"$184,091,184.83",34.198010,-118.261020
794,TX,Jefferson (County),"$162,696,152.11",31.087212,-104.271552
812,TX,Orange (County),"$124,538,402.56",31.087212,-104.271552
775,TX,Fort Bend (County),"$97,394,301.58",33.766503,-97.624685
354,MI,Wayne (County),"$91,868,928.48",43.942679,-82.996110
219,IL,Cook (County),"$90,293,293.25",42.307271,-89.902160
778,TX,Galveston (County),"$88,355,433.09",31.087212,-104.271552
136,FL,Miami-Dade (County),"$86,860,598.52",28.750540,-82.500100


In [29]:
#Enter Google Geocode Key
gkey = ""

In [31]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [32]:
#Save lat and lng as a dataframe
locations = county_summary_table_a[['lat','lng']].astype(float)
#Store humidity as array
approved_IHP_amount = county_summary_table_a['Total Approved IHP Amount'].astype(float)
#Set Humidity as weight
max_approved_IHP_amount = approved_IHP_amount.max()

In [33]:
#Customize the size of the figure
figure_layout = {
    'width': '700px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}


In [34]:
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights = approved_IHP_amount, dissipating=False, max_intensity=max_approved_IHP_amount, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…